<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E4%B8%89%E9%80%B1_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').sheet1

In [3]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [4]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [6]:
df.describe()

,Account,Date,Cleaned_Content
count,312,312,312
unique,272,13,312
top,peculab,"Sep 5, 2024",Alan960609 commented 我是花蓮高工 杜謙祺 會想上這門課是因為想學習程式...
freq,11,110,1


In [7]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 49.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=6df5b89aa3756d45b623b977fcd5f6ed5fa293b54585d9a51afc0cbcf4effab6
  Stored in directory: /root/.cache/pip/wheels/8a/0a/37/f15b8568f5463f1427466f701e9d3ba514035eb703f885efee
Successfully built snownlp


In [8]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


In [9]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.411 seconds.
DEBUG:jieba:Loading model cost 1.411 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [10]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [11]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [12]:
df['Chinese_Content']

,Chinese_Content
0,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...
1,高中生們兒早哇我是這次的助教我叫家瑋現為台灣師範大學歷史系三年級的同學也是個愛大便的小孩子這...
2,各位好我是鄭鈞賢是台師大機電系大三的從前我的夢想是成為一名頂尖的歌手可惜因為一些個人因素我唱...
3,各位好和大家一樣是高中生就讀北市數位實中同時在臺大資工擔任研究助理雖然如此但我總是宣稱自己是...
4,大家好我是胡程勛目前高二就讀彰化縣溪湖高中班在班上擔任的幹部是設備股長下面的程式是我在高一的...
...,...
307,我是嘉工陳信錩希望可以學到東西
308,我是南港高中高廷翰對程式基礎有幫助
309,我是鳳梨大學傳播系的漁人在網路上路過一起來學的以下程式碼可以生成圖片喔是我在課上來的哈
310,我是鳳梨大學傳播系的漁人在網路上路過一起來學的以下程式碼可以生成圖片喔是我在課上來的哈


In [13]:
df['Tokenized']

,Tokenized
0,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,..."
1,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現..."
2,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ..."
3,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,..."
4,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,..."
...,...
307,"[我, 是, 嘉工, 陳信錩, 希望, 可以, 學到, 東西]"
308,"[我, 是, 南港高中, 高廷翰, 對, 程式, 基礎, 有, 幫助]"
309,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."
310,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."


In [14]:
word_counts = Counter(all_words)
word_counts

Counter({'學校': 5,
         '姓名': 2,
         '以及': 3,
         '為什麼': 2,
         '來上': 4,
         '這門課': 47,
         '留下': 1,
         '第一': 9,
         '程式': 143,
         '連結': 16,
         '想要': 29,
         '透過': 12,
         '實現': 2,
         '心中': 1,
         '想法': 1,
         '高中生': 3,
         '這次': 2,
         '助教': 7,
         '家瑋': 3,
         '現為': 1,
         '台灣師範大學': 1,
         '歷史系': 1,
         '三年': 1,
         '同學': 3,
         '大便': 1,
         '小孩子': 1,
         '這是': 13,
         '大家': 145,
         '整理': 2,
         '蔡老師': 2,
         '可以': 54,
         '觀覽': 1,
         '一下': 6,
         '下面': 3,
         '那位': 1,
         '冷面': 1,
         '笑匠': 1,
         '小心': 1,
         '嘿然後有': 1,
         '時候': 3,
         '我會': 5,
         '不正': 2,
         '搜哩嘿': 1,
         '拍謝': 1,
         '黑若': 1,
         '不想': 1,
         '直接': 1,
         '電腦': 5,
         '一關': 1,
         '機此': 1,
         '失效': 1,
         '上面': 2,
         '各位': 6,
         '鄭鈞賢': 1,
    

In [15]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [16]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [17]:
word_freq_df

,Word,Frequency
0,高中,149
1,程式,143
2,希望,111
3,學習,62
4,可以,54
...,...,...
962,模型,1
963,弄出,1
964,哪天,1
965,工具,1


In [18]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 2s (3,202 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [19]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [20]:
# 按日期計算討論熱度
df['Date'] = pd.to_datetime(df['Date'])
heatmap_data = df.groupby('Date').size().reset_index(name='Discussion_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='Discussion_Count',
              title='不同時間段的討論熱度趨勢',
              labels={'Date': '日期', 'Discussion_Count': '討論數量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="討論數量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [22]:
from google.colab import userdata
apikey = userdata.get('GEMINI_API_KEY')

In [24]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言的重點和情緒表現可歸納如下：

**留言重點摘要：**

1.  **參與者背景多元，但高中生為主體：** 絕大多數留言者是高中生，來自不同學校，少數為大學生、老師或社會人士。這顯示課程受眾廣泛，尤其吸引高中生群體。
2.  **學習程式與AI的動機強烈：**
    *   **興趣導向：** 最常見的動機是對程式設計或AI本身有濃厚興趣，認為其「有趣」、「很酷」、「很炫」。
    *   **未來發展考量：** 許多學生意識到AI和程式設計是未來趨勢，對「未來有幫助」、「想從事科技業/工程師」、「想讀相關科系」、「增加學習歷程內容」、「精進實力配合時代」等抱有期望。
    *   **實際應用需求：** 一部分學生希望學會利用AI解決生活中的問題，例如「讓AI幫忙寫作業」、「讓生活變得更方便」、「賺錢」、「精進寫程式能力」。
    *   **提升自身能力：** 包括「精進自己編程能力」、「學習新知識」、「更深入了解」、「嘗試不同領域」。
    *   **老師/朋友推薦或學校安排：** 部分學生是因老師推薦或朋友邀請而來。
3.  **對課程的學習態度積極：** 許多人表達了「想學習」、「會努力」、「會加油」的態度，即使是零基礎也勇於嘗試。
4.  **技術基礎程度不一：** 大部分學生表示自己是「0基礎」、「程式小白」、「沒接觸過」，但也有少數人有Python基礎或曾修習相關課程。
5.  **積極利用與分享資源：** 許多留言附上自己或助教分享的Colab程式連結，顯示學習者間有相互學習與分享的習慣。助教也積極提供整理好的連結並鼓勵互動。

**重點中的情緒表現：**

在這些留言中，最為普遍和突出的情緒是 **「希望（Aspiration/Hope）」**。這種希望體現在對未來發展的憧憬、對新知識的渴望以及對自我提升的期許。

與「希望」相關聯的情緒和內容包括：

*   **渴望學習與探索（Curiosity & Eagerness to Learn）：** 「想學程式」、「想了解AI」、「想學習更多知識」、「想嘗試看看」、「探索不同領域」。這反映了學生們對未知領域的積極態度和求知慾。
*   **熱情與興趣（Enthusiasm & Interest）：** 許多人明確表示「對程式/AI很有興趣」、「覺得很有趣」、「很酷」、「充滿熱情